### Loading Packages and Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)

#import xarray as xr
import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws
from db_queries import get_population

In [2]:
ls /mnt/team/simulation_science/priv/engineering/vivarium_nih_us_cvd/seed-testing/alabama/2023_10_24_14_50_26/count_data

binned_ldl_exposure_time.csv     risk_exposure_time.csv
binned_ldl_exposure_time.hdf     risk_exposure_time.hdf
binned_sbp_exposure_time.csv     sbp_medication_person_time.csv
binned_sbp_exposure_time.hdf     sbp_medication_person_time.hdf
deaths.csv                       state_person_time.csv
deaths.hdf                       state_person_time.hdf
healthcare_visits.csv            transition_count.csv
healthcare_visits.hdf            transition_count.hdf
intervention_person_time.csv     ylds.csv
intervention_person_time.hdf     ylds.hdf
ldlc_medication_person_time.csv  ylls.csv
ldlc_medication_person_time.hdf  ylls.hdf


In [4]:
sim_results_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_nih_us_cvd/seed-testing/alabama/2023_10_24_14_50_26/count_data/'

# Loading and Cleaning Data

### Load Data

In [6]:
tran = pd.read_csv(sim_results_dir + 'transition_count.csv')

In [7]:
tran = tran.loc[tran.transition.isin(['susceptible_to_ischemic_heart_disease_and_heart_failure_to_acute_myocardial_infarction','heart_failure_from_ischemic_heart_disease_to_acute_myocardial_infarction_and_heart_failure','post_myocardial_infarction_to_acute_myocardial_infarction','susceptible_to_ischemic_heart_disease_and_heart_failure_to_heart_failure_from_ischemic_heart_disease','susceptible_to_ischemic_heart_disease_and_heart_failure_to_heart_failure_residual','post_myocardial_infarction_to_heart_failure_from_ischemic_heart_disease','susceptible_to_ischemic_stroke_to_acute_ischemic_stroke','chronic_ischemic_stroke_to_acute_ischemic_stroke'])]
tran['transition_new'] = np.where(tran.transition.isin(['susceptible_to_ischemic_heart_disease_and_heart_failure_to_acute_myocardial_infarction','heart_failure_from_ischemic_heart_disease_to_acute_myocardial_infarction_and_heart_failure','post_myocardial_infarction_to_acute_myocardial_infarction']), 'Myocardial Infarction', np.where(tran.transition.isin(['susceptible_to_ischemic_stroke_to_acute_ischemic_stroke','chronic_ischemic_stroke_to_acute_ischemic_stroke']),'Ischemic Stroke','Heart Failure'))
tran_US = tran.groupby(['input_draw','year','scenario','transition_new']).value.sum().reset_index()
tran_US.head()

,input_draw,year,scenario,transition_new,value
0,357,2021,baseline,Heart Failure,3463.0
1,357,2021,baseline,Ischemic Stroke,708.0
2,357,2021,baseline,Myocardial Infarction,1072.0
3,357,2021,lifestyle_100,Heart Failure,3463.0
4,357,2021,lifestyle_100,Ischemic Stroke,708.0


In [8]:
tran_US_baseline = tran_US.loc[tran_US.scenario == 'baseline']
tran_US_baseline = tran_US_baseline.rename(columns = {'value':'baseline'})
tran_US_total = tran_US.merge(tran_US_baseline[['input_draw','year','transition_new','baseline']], on=['input_draw','year','transition_new']) 
tran_US_total['averted_count'] = tran_US_total['baseline'] - tran_US_total['value']
tran_US_total['averted_percent'] = 100*((tran_US_total['baseline'] - tran_US_total['value'])/tran_US_total['baseline'])
tran_US_total.head()

,input_draw,year,scenario,transition_new,value,baseline,averted_count,averted_percent
0,357,2021,baseline,Heart Failure,3463.0,3463.0,0.0,0.0
1,357,2021,lifestyle_100,Heart Failure,3463.0,3463.0,0.0,0.0
2,357,2021,lifestyle_50,Heart Failure,3463.0,3463.0,0.0,0.0
3,357,2021,outreach_100,Heart Failure,3463.0,3463.0,0.0,0.0
4,357,2021,outreach_50,Heart Failure,3463.0,3463.0,0.0,0.0


In [14]:
tran_US_total.loc[(tran_US_total.averted_count < 0) & (tran_US_total.year == 2023)] 

,input_draw,year,scenario,transition_new,value,baseline,averted_count,averted_percent
46,357,2023,outreach_50,Heart Failure,3373.0,3372.0,-1.0,-0.029656
477,602,2023,lifestyle_100,Myocardial Infarction,1659.0,1658.0,-1.0,-0.060314
478,602,2023,lifestyle_50,Myocardial Infarction,1659.0,1658.0,-1.0,-0.060314
